# Interacting with external file

## Outline

1. **Working with files**<br>
Manipulating filesystem paths, creating list of files from filenames pattern matching, accessing files.
2. **Passing a text file into a python array**<br>
<span style="color:blue">*Numpy*</span> solution.
3. **Reading/Writing tables from/to a file**<br>
<span style="color:blue">**astropy.io.ascii**</span> package.
4. **Interacting with fits files**<br>
<span style="color:blue">**astropy.io.fits**</span> package.

## Exercise &mdash; overview

Starting from a group of data files, representing models of stellar evolutionary tracks, we are goint to:
- create new paths for dividing them in subgroups;
- modifing the content of each file
- read their content and store it in a python data structure, i.e. numpy array.
- write a table in a LaTeX format to a file
- create fits file out of the newly created tables 

## 1. Working with files

The functionalities of these python modules:
- <span style="color:blue">**os**</span> package for accessing the filesystem
- <span style="color:blue">**os.path**</span> for pathname manipulation
- <span style="color:blue">**glob**</span> for filename pattern matching
- the python built-in <span style="color:blue">**open()**</span> for reading/writing into text files
<br>
<br>
are all included in <span style="color:blue">**pathlib**</span> module!

### The <span style="color:blue">**pathlib**</span> module &mdash; object-oriented filesystem paths

From <span style="color:blue">**pathlib**</span> documentation:<br>
*"Manipulating filesystem paths as string objects can quickly become cumbersome: multiple calls to <span style="color:blue">**os.path.join()**</span> or <span style="color:blue">**os.path.dirname()**</span>, etc. This module offers a set of classes featuring all the common operations on paths in an easy, object-oriented way."*

This module has been included in the Python 3.4 standard library. You only need to install it for Python 3.3 or older.
<br>
Note: install pathlib2 for python 2.7

#### Basic use

Importing the classes.

In [ ]:
from pathlib import Path, PurePath

Filesystem paths initialization.

In [ ]:
p = Path.cwd()
p

In [ ]:
p.home()

Useful checks returning *True* or *False* are <span style="color:blue">**Path.is_dir()**</span>, <span style="color:blue">**Path.is_absolute()**</span>, <br><span style="color:blue">**Path.exists()**</span>, <span style="color:blue">**Path.is_file()**</span>, <span style="color:blue">**Path.is_symlink()**</span>, etc..

In [ ]:
p.is_absolute()

In [ ]:
q = Path()
q

In [ ]:
q.is_absolute()

Join the paths to create full path to files. Very useful for organizing files and folders!!

In [ ]:
p.joinpath('mydir')    # to create the path

In [ ]:
p.joinpath('mydir').mkdir(exist_ok=True)    # this create the directory at the pointed position
                                            # exist_ok parameter only from python 3.5

In [ ]:
p1 = p.joinpath('mydir', 'subdir1')
p2 = p.joinpath('mydir', 'subdir2')
p3 = p.joinpath('mydir', 'subdir3')
p1.mkdir(exist_ok=True)
p2.mkdir(exist_ok=True)
p3.mkdir(exist_ok=True)

Accessing parts of the path, parent path, filename and file suffix.

In [ ]:
p.parts

In [ ]:
print (p.parent)
print (p.parents[0])
print (p.parents[1])
print (p.parents[2])

In [ ]:
f = p.joinpath('inputoutput.ipynb')
f

In [ ]:
f.name

In [ ]:
f.stem

In [ ]:
f.suffix

Last but (definetely) not least: <span style="color:blue">**glob**</span> method.

In [ ]:
# We create 10 text files with the same content
for i in range(10) :
    t = p1.joinpath('file%s.txt'%i)
    t.write_text('Text file contents')
    print (t)
    print (t.read_text())

In [ ]:
# We then create a list of Path objects with glob
sorted(p1.glob('*.txt'))

In [ ]:
# You can always access the string part of Path object with str()
list_of_txt = [str(path) for path in sorted(p1.glob('*.txt'))]
print (list_of_txt)

#### Useful links

[https://docs.python.org/3/library/pathlib.html](https://docs.python.org/3/library/pathlib.html)
<br>
[https://www.python.org/dev/peps/pep-0428/](https://www.python.org/dev/peps/pep-0428/)

[https://pymotw.com/2/glob/](https://pymotw.com/2/glob/)

#### A practical example 

We use open() method and the list of paths created with glob to modify the text files and copy them to a new directory.

In [ ]:
from shutil import copy, move

for path in sorted(p1.glob('*.txt')) :
    newpath = path.parents[1].joinpath('subdir2',path.parts[-1])
    print (newpath)
    with path.open('a') as file :
        file.write('\nNew line')
    with path.open('r') as file :
        for line in file :
            line = line.strip()
            #print (line)
    copy(str(path),str(newpath))

Another solution for safely saving a new file without modifying the old one.

In [ ]:
# Rewriting the unmodified text files
for i in range(10) :
    t = p1.joinpath('file%s.txt'%i)
    t.write_text('Text file contents')
# Safely saving the modified text files
for path in sorted(p1.glob('*.txt')) :
    newpath = path.parents[1].joinpath('subdir3',path.parts[-1])
    print (newpath)
    with path.open('r') as infile, newpath.open('w') as outfile :
        for line in infile :
            cols = line.split()
            if cols[0]=='Text' :
                line = line.replace('Text','New text')
            outfile.write(line)

### Exercise &mdash; part 1

In *STARS_models* folder, stellar evolutionary models are stored for stars with different initial masses and metallicities. An example file is named data_z050_15 for a star at metallicity $Z=0.50$ and initial mass of 15 $M_\odot$. <br>
Manipulate the data files by first creating a path to the folder and then copy them in subdirectories for each metallicity. At the same time, modify the files to comment ('#') their header (first 35 lines).

## 2. Passing a text file into a python array 

<span style="color:blue">*Numpy*</span> (Numerical Python) is a package for scientific computing that is going to be covered in more detail in other lectures. Despite the scientific application, <span style="color:blue">*Numpy*</span> can be used to store generic data in multi-dimensional arrays. We make use only of the built-in numpy functions to read text files.

### The functions <span style="color:blue">**numpy.loadtxt**</span> and <span style="color:blue">**numpy.genfromtxt**</span>

Loading information in a matrix (multi-dimensional python array) with <span style="color:blue">**numpy.loadtxt**</span>.

In [ ]:
# Create the path to STARS models (you should have done already in Exercise part 1)
stars = p.joinpath('STARS_models')
z050_15 = stars.joinpath('data_z050_15')
z050_15

In [ ]:
import numpy as np

# Load the text skipping the header (you don't need it if you have solved Execrise part 1)
mat = np.loadtxt (str(z050_15), skiprows=35)

We can specify the data columns we want to load into our array.

In [ ]:
arr = np.loadtxt (str(z050_15), skiprows=35, usecols=(1,))    # one-dimensional array
mat = np.loadtxt (str(z050_15), skiprows=35, usecols=(1,2,3,4,6))   # we loaded: age, 
                                                                    # solar radii, temperature,
                                                                    # luminosity, solar masses

Accessing the one-dimensional arrays from the multi-dimensional one or ...

In [ ]:
age = mat[:,0]
radius = mat[:,1]
temp = mat[:,2]
lum = mat[:,3]
mas = mat[:,4]

... unpack directly with <span style="color:blue">**numpy.loadtxt**</span>

In [ ]:
age, radius, temp, lum, mas = np.loadtxt (str(z050_15), skiprows=35, usecols=(1,2,3,4,6), unpack = True)

<span style="color:blue">**numpy.loadtxt**</span> tries to interprete the data type.

In [ ]:
type(age)

In [ ]:
age[0].dtype

If data types are different, we can either force <span style="color:blue">**numpy.loadtxt**</span> to load in string format or use <span style="color:blue">**numpy.genfromtxt**</span>

[https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.loadtxt.html](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.loadtxt.html)

[https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.genfromtxt.html](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.genfromtxt.html)

### Exercise &mdash; part 2

Load from model *data_z020_10* only surface temperature and solar luminosities in string format, by unpacking directly the columns into two one-dimensional arrays. 

**NOTE**<br>
**The package <span style="color:blue">*Astropy*</span> will be covered in another lecture.<br>
We now (briefly) introduce how to read/write tables in text and fits files.**

## 3. Reading/Writing tables from/to a file

A method to read and write tables in a wide range of formats is with <span style="color:blue">**astropy.io.ascii**</span>.

With <span style="color:blue">**astropy.io.ascii**</span> you can define data start point, header, delimiter, etc. in a very flexible way. Look at<br>
[http://docs.astropy.org/en/stable/io/ascii/](http://docs.astropy.org/en/stable/io/ascii/)

#### Basic use

In [ ]:
from astropy.io import ascii

x = np.array([1, 2, 3])
y = x ** 2
ascii.write([x, y], 'values.dat', names=['x', 'y'])

In [ ]:
data = ascii.read('values.dat')
print (data)

In [ ]:
data.show_in_notebook()

In [ ]:
ascii.write(data, format='latex')

### Exercise &mdash; part 3

Write the table from Exercise part 2 to a file, in LaTeX format.

## 4. Interacting with fits files

With the package <span style="color:blue">**astropy.io.fits**</span> we can access FITS (Flexible Image Transport System) files, read/modify their headers and main bodies (data).

[http://docs.astropy.org/en/stable/io/fits/](http://docs.astropy.org/en/stable/io/fits/)

We see a pracical example where a <span style="color:blue">**BinTableHDU**</span> class ([http://docs.astropy.org/en/stable/io/fits/api/tables.html](http://docs.astropy.org/en/stable/io/fits/api/tables.html)) will be instantiated, out of the table created in Section 3.

In [ ]:
from astropy.io import fits

BinHDU = fits.BinTableHDU(data, name='DATA')
print (BinHDU)

In [ ]:
print (repr(BinHDU.header))    # repr() to display the entire header as it appears in the FITS file

In [ ]:
print (BinHDU.data)

Writing out the Bin Table.

In [ ]:
help(fits.writeto)

In [ ]:
fits.writeto('my_fits_table.fits', np.array(data), overwrite=True)    # data must be an array!
fits.writeto('my_fits_table.fits', BinHDU.data, overwrite=True)       # or FITS_rec

As last example, we read the newly created fits file and write into the header

In [ ]:
fitspath = Path('my_fits_table.fits')
hdu = fits.open(fitspath)

In [ ]:
hdu.info()

If we need the extension name

In [ ]:
fits.writeto('my_fits_table.fits', BinHDU.data, header=BinHDU.header, overwrite=True)
hdu = fits.open(fitspath)
hdu.info()

Reading/Writing the header cards

In [ ]:
hdr = hdu[1].header
list(hdr.keys())

In [ ]:
hdr['EXTNAME']

In [ ]:
hdr['history'] = 'Test writing on January, 8th 2018'
print (repr(hdr))

In [ ]:
hdr.set('EXTNAME2', value='MYDATA', comment='Card created during ESOpy2.0', after='EXTNAME')
print (repr(hdr))

### Exercise &mdash; part 4

Create a fits file out of the table of Exercise part 2 